In [66]:
import spkit as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from copy import deepcopy
from mne.preprocessing import compute_proj_ecg
from mne_connectivity import envelope_correlation

In [67]:
#mne.set_config("MNE_USE_CUDA", True)
#print(mne.get_config("MNE_USE_CUDA"))
print(mne.get_config("MNE_USE_CUDA"))

True


In [68]:
data = np.array(pd.read_csv("D:\Clean Data\IGE\shirband\samples.csv", skiprows=2))
data = np.delete(data, [0, 1, -1, -3, -4], axis=1)

In [69]:
#ch_name = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'A1', 'A2', 'Fz', 'Cz', 'Pz', 'T2', 'T1', 'ECG']
channel_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'A1', 'A2', 'Fz', 'Cz', 'Pz', 'ECG']
#ch_name = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'A1', 'A2', 'Fz', 'Cz', 'Pz', 'T2', 'T1']
np.shape(channel_names)

(22,)

In [70]:
#data = data[1000000:1000300]
np.shape(data)

(1815746, 22)

In [71]:
X = deepcopy(data)

In [72]:
sfreq = 300
info = mne.create_info(channel_names, sfreq, ch_types=['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'ecg'])
raw = mne.io.RawArray(X.T, info)
raw.set_montage('standard_1020')

Creating RawArray with float64 data, n_channels=22, n_times=1815746
    Range : 0 ... 1815745 =      0.000 ...  6052.483 secs
Ready.


<RawArray | 22 x 1815746 (6052.5 s), ~304.8 MB, data loaded>

In [73]:
epochs = mne.make_fixed_length_epochs(raw, duration=30, preload=True)


Not setting metadata
201 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 201 events and 9000 original time points ...
0 bad epochs dropped


In [78]:
scaler = mne.decoding.Scaler(info, scalings='median', with_mean=False, with_std=True)
#scaler.fit_transform(epochs)

In [80]:
#train_x = deepcopy(epochs.get_data())
newx = scaler.fit_transform(epochs.get_data())
np.min(newx)

-19.08034470354325